# Bedtools Tutorial

In [1]:
import os, sys
import pandas as pd

# .bed file format

basic format: https://genome.ucsc.edu/FAQ/FAQformat.html#format1

- tab separated
- three main columns - chr, start, stop

# indexing

- Tutorial: https://genome-blog.soe.ucsc.edu/blog/2016/12/12/the-ucsc-genome-browser-coordinate-counting-systems/

- Start = starting position of region
- Stop = stopping position of region
- Size = n bases (STOP - START)
- Range = sequence limits.

##### Indexing matters for determining the "real" region
- Index from zero or one?
- Include/exclude STOP?


- ALWAYS include START

## .bed files are 0-start, half-open (0-based)

- Index from zero (count: "0, 1, 2, 3, 4")
- Size = stop - start

- half-open (region spanning 0-5: INCLUDES 0, EXCLUDES 5, SIZE = 5-0, RANGE = 0-5) 

## UCSC genome browser coordinates are 1-start, fully-closed

- Index from one (count: "1,2,3,4,5")
- Size = stop - start + 1

- fully-closed (region spanning from 1-5: INCLUDES 1, INCLUDES 5, SIZE = 5-1

# A word about SNPs

Table 2. SNP coordinates in UCSC web browser (1-start) vs table (0-start)

                               rs782519173 (hg38)	      Start	     End
    Positioned in web browser: 1-start, fully-closed	   133255708	 133255708
    Stored in table:           0-start, half-open	      133255707	 133255708
                    
###### What is the coordinate of the SNP? 

# Example files 

In [2]:
# make two bed files

dfa = pd.DataFrame({
                    "#chr": ["chr1", "chr1"], 
                    "start": [1, 25],
                    "end":[10, 50]
                  })

dfb = pd.DataFrame({
                    "#chr": ["chr1","chr1", "chr1"], 
                    "start": [1, 5, 50],
                    "end":[10, 25, 100]
                  })
# write bed files
dfa.to_csv('a.bed', sep = '\t', index = False)
dfb.to_csv('b.bed', sep = '\t', index = False)

## a.bed

In [3]:
dfa

,#chr,start,end
0,chr1,1,10
1,chr1,25,50


## b.bed

In [4]:
dfb

,#chr,start,end
0,chr1,1,10
1,chr1,5,25
2,chr1,50,100


# Intersect

- find overlapping sequences from two different .bed files. 

https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html

## overlapping

In [6]:
%%bash

bedtools intersect -a a.bed -b b.bed 

chr1	1	10
chr1	5	10


### what just happened?
- from a.bed
    - chr1:1-10 overlaps chr1:1-10 in b.bed
        - returns overlap of chr1:1-10
    - chr1:1-10 overlaps chr1:5-25 in b.bed
        - returns overlap of chr1:5-10 
        
        
    - chr1:25-50 does NOT overlap chr1:1-10, chr1:5-25, chr1:50-100  in b.bed
        - no return of chr1:25-50

## non-overlapping

- find non-overlapping sequences


        "-v"

In [7]:
%%bash

bedtools intersect -a a.bed -b b.bed -v

chr1	25	50


### what just happened?
- from a.bed
    - chr1:1-10 overlaps chr1:1-10 in b.bed
        - No return of overlap of chr1:1-10
    - chr1:1-10 overlaps chr1:5-25 in b.bed
        - No return of overlap of chr1:5-10 
        
        
    - chr1:25-50 does NOT overlap chr1:1-10, chr1:5-25, chr1:50-100  in b.bed
        - Returns chr1:25-50

### opposite input order

In [8]:
%%bash
# bedtools subtract -a b.bed -b a.bed

bedtools intersect -a b.bed -b a.bed -v

chr1	50	100


## partially overlapping

"-f" Minimum overlap required as a fraction of A. Default is 1E-9 (i.e. 1bp).

"-F" Minimum overlap required as a fraction of B. Default is 1E-9 (i.e., 1bp).

##### 80% of region in a.bed must overlap region in b.bed

In [9]:
%%bash

bedtools intersect -a a.bed -b b.bed -f 0.8  # require 80% overlap of any region in a.bed 

chr1	1	10


### what happened? 
- a.bed has two regions(chr1:1-10 , chr1:5-10)


- Region1: chr1:1-10 
        in a.bed, 100% of chr1:1-10 overlaps chr1:1-10 in b.bed
        
        100% > 80% - This region was returned
        
- Region2: chr1:5-10
        in a.bed, 50% of chr1:5-10 overlaps chr1:5-25 in b.bed
        
        50% < 80% - This region was not returned

##### 50% of region in a.bed must overlap region in b.bed

In [11]:
%%bash

bedtools intersect -a a.bed -b b.bed -f 0.5  # require 50% overlap of any region in a.bed 

chr1	1	10
chr1	5	10


# Jaccard index 

- Goal: Quantify overlap between a.bed and b.bed


- Jaccard index = n intersected bases / n union bases

In [15]:
%%bash 

bedtools jaccard -a a.bed -b b.bed c.bed d.bed

intersection	union	jaccard	n_intersections
9	99	0.0909091	1


## What happened?

- intersection: 9 bases overlap a.bed and b.bed


- union: 99 bases total in a.bed + b.bed


- Jaccard index = 0.0909091 = 9/99


- n_intersections = 1 (a.bed x b.bed)

# Genome coverage

Goal: quantify genome coverage of .bed file

Get genome files from 
- hg19: http://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/ 
- hg38: http://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/

##### long run time!

In [2]:
%%bash

bedtools genomecov -i a.bed -g hg19.chrom.sizes 

chr1	0	249250587	249250621	1
chr1	1	34	249250621	1.36409e-07
chr2	0	243199373	243199373	1
chr3	0	198022430	198022430	1
chr4	0	191154276	191154276	1
chr5	0	180915260	180915260	1
chr6	0	171115067	171115067	1
chr7	0	159138663	159138663	1
chrX	0	155270560	155270560	1
chr8	0	146364022	146364022	1
chr9	0	141213431	141213431	1
chr10	0	135534747	135534747	1
chr11	0	135006516	135006516	1
chr12	0	133851895	133851895	1
chr13	0	115169878	115169878	1
chr14	0	107349540	107349540	1
chr15	0	102531392	102531392	1
chr16	0	90354753	90354753	1
chr17	0	81195210	81195210	1
chr18	0	78077248	78077248	1
chr20	0	63025520	63025520	1
chrY	0	59373566	59373566	1
chr19	0	59128983	59128983	1
chr22	0	51304566	51304566	1
chr21	0	48129895	48129895	1
chr6_ssto_hap7	0	4928567	4928567	1
chr6_mcf_hap5	0	4833398	4833398	1
chr6_cox_hap2	0	4795371	4795371	1
chr6_mann_hap4	0	4683263	4683263	1
chr6_apd_hap1	0	4622290	4622290	1
chr6_qbl_hap6	0	4611984	4611984	1
chr6_dbb_hap3	0	4610396	4610396	1
chr17_ctg5_hap1	0	1680828	1680828	1

### What happened? 

#### per chromosome coverage
- col 0 - chromosome (or entire genome)
- col 1 - depth of coverage from features in input file (0 = not covered, 1 = covered)
- col 2 - number of bases on chromosome (or genome) with depth equal to column 2.
- col 3 - size of chromosome (or entire genome) in base pairs
- col 4 - fraction of bases on chromosome (or entire genome) with depth equal to column 2.


        chr    depth   nbases w/depth   chr/genome size   fraction w/depth
        
        chr1	0	249250587	249250621	1
        chr1	1	34	249250621	1.36409e-07
        chr2	0	243199373	243199373	1
        chr3	0	198022430	198022430	1


#### whole genome coverage
    genome	0	3137161230	3137161264	1
    genome	1	34	3137161264	1.08378e-08

# Shuffle

- Goal: Create expectation dataset, find random regions of genome w/ same length


- Inputs
    - "-i" .bed file
    - "-g" genome file (e.g. "/dors/capra_lab/data/dna/human/hg19/hg19_trim.chrom.sizes")

In [21]:
%%bash

bedtools shuffle -i a.bed -g hg19.chrom.sizes 

chr10	67454738	67454747
chr9	138402130	138402155


### What happened? 

- Shuffling a.bed returns two random regions from genome. 
- Shuffled region 1 is 9 bp long, matches chr1:1-10
- Shuffled region 2 is 25 bp long, matches chr1:25-50

##### favorite args

- "-chrom" shuffle for regions on same chromosome


- "-excl" exclude regions like gaps, coding regions, repeat regions from shuffle


- "noOverlapping" shuffled regions should not overlap

In [23]:
%%bash

bedtools shuffle -i a.bed -g hg19.chrom.sizes -chrom  -noOverlapping # only chr1 

chr1	63136898	63136907
chr1	147087665	147087690


## pybedtools

Python extension for BEDTools

https://daler.github.io/pybedtools/main.html

## ML's pybedtools script for shuffling

In [ ]:
import os
import sys, traceback
import argparse
from datetime import datetime
import numpy as np
from functools import partial
from multiprocessing import Pool
from pybedtools import BedTool


"""
ARGUMENTS

    input.bed
    sample_id
    N shuffle iterations
    species
    N threads
    output.bed
"""

arg_parser = argparse.ArgumentParser(description="shuffle a bed file")

arg_parser.add_argument("input_bed", help='bed file 1 w/ full path')

arg_parser.add_argument("sample_id", help='str label for files')

arg_parser.add_argument("-i", "--iters", type=int, default=100,
                        help='number of simulation iterations; default=100')

arg_parser.add_argument("-s", "--species", type=str, default='hg19', choices=['hg19', 'hg38', 'mm10'],
                        help='species and assembly; default=hg19')

arg_parser.add_argument("-n", "--num_threads", type=int,
                        help='number of threads; default=SLURM_CPUS_PER_TASK or 1')

arg_parser.add_argument("--print_counts_to", type=str, default=None,
                        help="print expected counts to file")

args = arg_parser.parse_args()


""" 
Sort arguments into variables
"""

TEST_BED = args.input_bed
SAMPLE_ID = args.sample_id
COUNT_FILENAME = args.print_counts_to
ITERATIONS = args.iters
SPECIES = args.species
TEST_PATH = "/".join(TEST_BED.split("/")[:-1])

"""
These variables require information from arguments and variables above. 
"""
AGE_OUTFILE = f"{TEST_PATH}/{SAMPLE_ID}_enh_ages.bed"
SHUFFLE_ID = f"shuf-{SAMPLE_ID}"

"""
# calculate the number of threads
"""

if args.num_threads:
    num_threads = args.num_threads
else:
    num_threads = int(os.getenv('SLURM_CPUS_PER_TASK', 1))

In [ ]:
"""
#   functions
"""

def loadConstants(species):  
    
    """
    genome_dict[genome]:(genome_blacklist, genome_file) 
    """
    
    genome_dict = {
            'hg19': ("/dors/capra_lab/users/bentonml/data/dna/hg19/hg19_blacklist_gap.bed", "/dors/capra_lab/data/dna/human/hg19/hg19_trim.chrom.sizes"),
            'hg38': ("/dors/capra_lab/users/bentonml/data/dna/hg38/hg38_blacklist_gap.bed", "/dors/capra_lab/data/dna/human/hg38/hg38_trim.chrom.sizes"),
            'mm10': ("/dors/capra_lab/users/bentonml/data/dna/mm10/mm10_blacklist_gap.bed", "/dors/capra_lab/data/dna/mouse/mm10/mm10_trim.chrom.sizes")
            }
    
    return genome_dict[species]

### Shuffle function w/ pybedtools

In [ ]:
def calculateExpected(test_enh, test_path, species, shuffle_id):
    """
    1. Get blacklist regions, genome file. 
        genome file is not needed w/ pybedtools
        
    2. Make output path for shuffled files. 
    
    3. Do pybedtools shuffle w/ args
        genome = 'hg19'
        exclude blacklist
        maintain chromosome distribution
        no overlapping shuffled regions. 
        
    4. save the shuffle object as .bed file
        
    """
    
    # step 1 
    BLACKLIST, CHROM_SZ = loadConstants(species)  # note CHROM_SZ not needed w pybedtools

    # step 2
    shuffle_path = f"{test_path}/shuffle"  # make a shuffle path file
    os.system(f"mkdir {shuffle_path}")
    
    # step 3
    rand_file = BedTool(test_enh).shuffle(genome='hg19', excl=BLACKLIST, chrom=True, noOverlapping=True) # shuffle bed
    
    # step 4
    rand_out = f'{shuffle_path}/rand_file_{shuffle_id}.bed' # make shuffle file

    rand_file.saveas(rand_out)  # write shuffle file

### Shuffle function w/ command line

In [ ]:
def calculateExpected(test_enh, sample_id, test_path, species, iters):
    """
    1. Get blacklist regions, genome file. 
        
    2. Make output path for shuffled files. 
        
    4. Make file to save the shuffle object as .bed file
    
    3. Do pybedtools shuffle w/ args
        genome = 'hg19'
        exclude blacklist
        maintain chromosome distribution
        no overlapping shuffled regions. 
        
    """
    
    # step 1 
    BLACKLIST, CHROM_SZ = loadConstants(species)  

    # step 2
    shuffle_path = f"{test_path}/shuffle"  # make a shuffle path file

    if os.path.exists(shuffle_path) == False:

          os.system(f"mkdir {shuffle_path}")

    # step 3 
    shuffle_id = f"shuf-{sample_id}-{iters}"

    shuffle_out = os.path.join(shuffle_path, f'{shuffle_id}.bed')

    # step 4
    BEDshuf = f"bedtools shuffle \
                -i {test_enh} \ 
                -g {CHROM_SZ} \
                -excl {BLACKLIST} \
                -chrom \
                -noOverlapping \
                -maxTries 5000 \
                > {shuffle_out}"
    
    # run on commandline
    os.system(BEDshuf)  

In [ ]:
"""
# main
"""

def main(argv):
    print('python {:s} {:s}'.format(' '.join(sys.argv), str(datetime.now())[:20]))

    # create pool and run simulations in parallel
    pool = Pool(num_threads)
    partial_calcExp = partial(calculateExpected,\ 
                              BedTool(TEST_BED), TEST_PATH, SPECIES,\
                              (SHUFFLE_ID for SHUFFLE_ID in np.arange(ITERS))

if __name__ == "__main__":
    main(sys.argv[1:])

# bedtools suite
https://bedtools.readthedocs.io/en/latest/content/bedtools-suite.html